<a href="https://colab.research.google.com/github/Rameshkumar789/Computational-Menthods-INFO-5731/blob/main/Korlakunta_In_class_exercise_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [ ]:
from google.colab import files
import pandas as pd
import polyglot
from polyglot.text import Text, Word
import collections
from collections import Counter

In [ ]:
uploaded = files.upload()

Saving Tweets_data.csv to Tweets_data.csv


In [ ]:
df = pd.read_csv('Tweets_data.csv')
df.head()

,Document_Id,Clean_Text,Sentiment
0,1,got covidvaccin today first dose one go ill fu...,Positive
1,2,i elig covid19 vaccin i get first shot my seco...,Positive
2,3,how know one right covidvaccin httpstcov2ydw8ggw9,Neutral
3,4,thank wcsedu nurs staff made vaccin distribut ...,Positive
4,5,mentionitaii me regard two dose pfizer vaccin ...,Positive


In [ ]:
sent_word=[]
for i in range(len(df)):
  sent=Text(df['Clean_Text'][i])
  temp=sent.words
  for i in (temp):
    #print(i,i.polarity)
    if(i.polarity!=0):
      sent_word.append(i)

In [ ]:
count = collections.Counter(sent_word)

In [ ]:
tot_count = pd.DataFrame.from_dict(count, orient='index').reset_index()
tot_count.columns=['Words', 'Count']
tot_count.sort_values(by='Count', ascending=False)

,Words,Count
2,thank,7
10,like,3
13,good,3
15,proud,2
17,miss,2
16,death,2
23,recommend,2
43,protect,2
11,plea,2
9,hurt,2


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [ ]:
# Write your code here
from textblob import TextBlob
from sklearn.metrics import f1_score, accuracy_score
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
for i in range(len(df)):
  if(df['Sentiment'][i]=="Positive"):
    df.loc[i,['sentiment_val']]=1
  elif(df['Sentiment'][i]=="Neutral"):
    df.loc[i,['sentiment_val']]=0
  elif(df['Sentiment'][i]=="Negative"):
    df.loc[i,['sentiment_val']]=-1

In [ ]:
df.head()

,Document_Id,Clean_Text,Sentiment,sentiment_val
0,1,got covidvaccin today first dose one go ill fu...,Positive,1
1,2,i elig covid19 vaccin i get first shot my seco...,Positive,1
2,3,how know one right covidvaccin httpstcov2ydw8ggw9,Neutral,0
3,4,thank wcsedu nurs staff made vaccin distribut ...,Positive,1
4,5,mentionitaii me regard two dose pfizer vaccin ...,Positive,1


### TextBlob

In [ ]:
textblob_sent=[]
for i in range(len(df)):
  sent_temp = TextBlob(df['Clean_Text'][i]).sentiment.polarity
  if(sent_temp>0.0):
      textblob_sent.append(1)
  elif(sent_temp==0.0):
      textblob_sent.append(0)
  elif(sent_temp<0.0):
      textblob_sent.append(-1)

acc = accuracy_score(textblob_sent, df['sentiment_val'])*100
f1 = f1_score(textblob_sent, df['sentiment_val'], average='macro')*100

print("Accuracy: {}\nMacro F1-score: {}".format(acc, f1))

Accuracy: 42.0
Macro F1-score: 38.86247086247086


### VANDER

In [ ]:
vander_sent=[]
vader = SentimentIntensityAnalyzer()
for i in range(len(df)):
  temp_sent=vader.polarity_scores(df['Clean_Text'][i])['compound']
  #print(vader.polarity_scores(df['Clean_Text'][i]))
  if(temp_sent>0.0):
      vander_sent.append(1)
  elif(temp_sent==0.0):
      vander_sent.append(0)
  elif(temp_sent<0.0):
      vander_sent.append(-1)


acc = accuracy_score(vander_sent, df['sentiment_val'])*100
f1 = f1_score(vander_sent, df['sentiment_val'], average='macro')*100

print("Accuracy: {}\nMacro F1-score: {}".format(acc, f1))

Accuracy: 48.0
Macro F1-score: 47.73438547541816


### Support Vector Machine

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge',penalty='l2',alpha=1e-3,random_state=42,max_iter=100,learning_rate='optimal',tol=None,)),])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['Clean_Text'], df['sentiment_val'])

In [ ]:
model_predict=[]
learner = pipeline.fit(x_train, y_train)
# Predict class labels using the learner and output DataFrame
pred = learner.predict(x_test)

In [ ]:
model_predict=pred.tolist()

In [ ]:
model_predict

[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
acc = accuracy_score(model_predict, y_test)*100
f1 = f1_score(model_predict, y_test, average='macro')*100

print("Accuracy: {}\nMacro F1-score: {}".format(acc, f1))

Accuracy: 48.0
Macro F1-score: 33.33333333333333


In [ ]:
# Comparing above Evaluation metrics for different models, We can concluded that VANDER has better performed than other two methods.    